In [7]:
%reset -f

In [8]:
from google.colab import drive
drive.mount('/content/drive')
import ee
ee.Authenticate()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


True

In [4]:
import os
import ee

ee.Initialize(project='ee-andrewfullhart')

ic = ee.ImageCollection('NASA/NEX-GDDP')

inFILE = '/content/drive/My Drive/Colab Notebooks/Script Input Files/GHCN_Historical_Annual_Results.csv'

study_area = ee.FeatureCollection('users/andrewfullhart/SW_Study_Area')

ndays_months = ee.List([31., 28.25, 31., 30., 31., 30., 31., 31., 30., 31., 30., 31.])
order_months = ee.List([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])


"""Append nothing to output file name if running this block."""

with open(inFILE) as f:
  lines = f.readlines()

points_ft_list = ee.List([])
points_label_list = ee.List([])
for line in lines[1:]:
  row = line.strip('\n').split(',')
  stationID = ee.String(row[0])
  lon = float(row[1])
  lat = float(row[2])
  ft = ee.Feature(None, {'stationID':stationID}).setGeometry(ee.Geometry.Point(lon, lat))
  points_ft_list = points_ft_list.add(ft)
  points_label_list = points_label_list.add(stationID)

points_fc = ee.FeatureCollection(points_ft_list)


#"""Append _CLINET to output file name if running this block."""

#points_ft_list = ee.List([])
#points_label_list = ee.List([])
#parFOLDER = '/content/drive/My Drive/Colab Notebooks/CLIGEN/2015parfiles/2015parfiles/2015parfiles'
#par_files = os.listdir(parFOLDER)
#for parf in par_files:
#  if parf[:2] in ['az', 'nv', 'nm', 'ut', 'US']:
#    stationID = parf.strip('.par')
#    if stationID[0] == 'z':
#       stationID = 'a' + stationID
#    with open(os.path.join(parFOLDER, parf)) as f:
#      lines = f.readlines()
#    for line in lines:
#      label = line[:8]
#      label = label.rstrip().lstrip()
#      dataline = line[8:]
#      if label == 'LATT=':
#        lat = float(line.split('=')[1].strip('LONG').strip())
#        lon = float(line.split('=')[2].strip('YEARS').strip())
#    ft = ee.Feature(None, {}).setGeometry(ee.Geometry.Point(lon, lat))
#    points_ft_list = points_ft_list.add(ft)
#    points_label_list = points_label_list.add(stationID)

#points_fc = ee.FeatureCollection(points_ft_list)


start_year_global = 1974
end_year_global = 2013
years_list_global = ee.List.sequence(start_year_global, end_year_global)
years_index_global = ee.List.sequence(0, years_list_global.size().subtract(1))

def pointlabel_fn(point):
  point = ee.Feature(point).geometry()
  a_strng = ee.Number(point.coordinates().get(0)).format('%.3f')
  b_strng = ee.String('_')
  c_strng = ee.Number(point.coordinates().get(1)).format('%.3f')
  point_strng = a_strng.cat(b_strng).cat(c_strng)
  point_strng = point_strng.replace('\\.', '_', 'g').slice(1)
  return point_strng

point_labels_global = points_fc.toList(points_fc.size()).map(pointlabel_fn)

start = ee.Date.fromYMD(ee.Number(start_year_global), 1, 1)
end = ee.Date.fromYMD(ee.Number(end_year_global).add(1), 1, 1)
year_ic = ic.filterDate(start, end).select('pr')
modelfilter = ee.Filter.Or(
              ee.Filter.eq('scenario', 'historical'),
              ee.Filter.eq('scenario', 'rcp45'))
year_ic = year_ic.filter(modelfilter)
year_ic = year_ic.filter(ee.Filter.eq('model', 'CCSM4'))

def point_fn(point):
  stationID = ee.String(point.get('stationID'))
  point = point.geometry()
  data_list = year_ic.getRegion(point, 500).slice(1)

  def feature_fn(l):
    l = ee.List(l)
    date_str = ee.String(ee.String(l.get(0)).split('_').get(-1))
    year = ee.Number(date_str.slice(0, 4))
    month = ee.Number(date_str.slice(4, 6))
    prcp = ee.Number(l.get(4))
    prop_dict = {'year':year, 'month':month, 'precip':prcp}
    point_ft = ee.Feature(None, prop_dict)
    return point_ft

  data_fc = ee.FeatureCollection(data_list.map(feature_fn))
  nonzero_fc = data_fc.filter(ee.Filter.gt('precip', 0))
  number_avg = ee.Number(nonzero_fc.reduceColumns(ee.Reducer.mean(), ['precip']).get('mean')).multiply(86400)
  number_avg = ee.Number(ee.Algorithms.If(nonzero_fc.size().gt(0), number_avg, 0))
  number_sdv = ee.Number(nonzero_fc.reduceColumns(ee.Reducer.stdDev(), ['precip']).get('stdDev')).multiply(86400)
  number_sdv = ee.Number(ee.Algorithms.If(nonzero_fc.size().gt(0), number_sdv, 0))
  number_skw = ee.Number(nonzero_fc.reduceColumns(ee.Reducer.skew(), ['precip']).get('skew'))
  number_skw = ee.Number(ee.Algorithms.If(number_sdv.eq(0), 0, number_skw))
  #number_skw = ee.Number(ee.Algorithms.If(number_sdv.eq(1), 0, number_skw))
  number_skw = ee.Number(ee.Algorithms.If(nonzero_fc.size().gt(0), number_skw, 0))
  number_ft = ee.Feature(None, {'point':stationID,'mean':number_avg, 'sdev':number_sdv, 'skew':number_skw})
  return number_ft

out_fc_agg = ee.FeatureCollection(points_fc.map(point_fn))

task_agg = ee.batch.Export.table.toDrive(collection=out_fc_agg,
                           description='NEX_Historical_Annual_Trends_Agg',
                           folder='GEE_Downloads')

task_agg.start()



def year_fn(year):
  year = ee.Number(year)
  start = ee.Date.fromYMD(ee.Number(year), 1, 1)
  end = ee.Date.fromYMD(ee.Number(year).add(1), 1, 1)
  year_ic = ic.filterDate(start, end).select('pr')
  modelfilter = ee.Filter.Or(
                ee.Filter.eq('scenario', 'historical'),
                ee.Filter.eq('scenario', 'rcp45'))
  year_ic = year_ic.filter(modelfilter)
  year_ic = year_ic.filter(ee.Filter.eq('model', 'CCSM4'))


  def point_fn(point):
    point = point.geometry()
    a_strng = ee.Number(point.coordinates().get(0)).format('%.3f')
    b_strng = ee.String('_')
    c_strng = ee.Number(point.coordinates().get(1)).format('%.3f')
    point_strng = a_strng.cat(b_strng).cat(c_strng)
    point_strng = point_strng.replace('\\.', '_', 'g').slice(1)
    data_list = year_ic.getRegion(point, 500).slice(1)

    def feature_fn(l):
      l = ee.List(l)
      date_str = ee.String(ee.String(l.get(0)).split('_').get(-1))
      year = ee.Number(date_str.slice(0, 4))
      month = ee.Number(date_str.slice(4, 6))
      prcp = ee.Number(l.get(4))
      prop_dict = {'year':year, 'month':month, 'precip':prcp}
      point_ft = ee.Feature(None, prop_dict)
      return point_ft

    data_fc = ee.FeatureCollection(data_list.map(feature_fn))
    nonzero_fc = data_fc.filter(ee.Filter.gt('precip', 0))
    number_avg = ee.Number(nonzero_fc.reduceColumns(ee.Reducer.mean(), ['precip']).get('mean')).multiply(86400)
    number_avg = ee.Number(ee.Algorithms.If(nonzero_fc.size().gt(0), number_avg, 0))
    number_sdv = ee.Number(nonzero_fc.reduceColumns(ee.Reducer.stdDev(), ['precip']).get('stdDev')).multiply(86400)
    number_sdv = ee.Number(ee.Algorithms.If(nonzero_fc.size().gt(0), number_sdv, 0))
    number_skw = ee.Number(nonzero_fc.reduceColumns(ee.Reducer.skew(), ['precip']).get('skew'))
    number_skw = ee.Number(ee.Algorithms.If(number_sdv.eq(0), 0, number_skw))
    #number_skw = ee.Number(ee.Algorithms.If(number_sdv.eq(1), 0, number_skw))
    number_skw = ee.Number(ee.Algorithms.If(nonzero_fc.size().gt(0), number_skw, 0))
    number_ft = ee.Feature(None, {'point':stationID,'mean':number_avg, 'sdev':number_sdv, 'skew':number_skw})
    return number_ft

  means_fc = ee.FeatureCollection(points_fc.map(point_fn))
  return means_fc

means_fc_list = ee.FeatureCollection(years_list_global.map(year_fn)).toList(years_list_global.size())

def flatten_fn(mean_fc):
  mean_fc = ee.FeatureCollection(mean_fc)
  data_list_mean = ee.List(mean_fc.reduceColumns(ee.Reducer.toList(), ['mean']).get('list'))
  data_list_sdev = ee.List(mean_fc.reduceColumns(ee.Reducer.toList(), ['sdev']).get('list'))
  data_list_skew = ee.List(mean_fc.reduceColumns(ee.Reducer.toList(), ['skew']).get('list'))
  prop_dict_mean = ee.Dictionary(points_label_list.zip(data_list_mean).flatten())
  prop_dict_sdev = ee.Dictionary(points_label_list.zip(data_list_sdev).flatten())
  prop_dict_skew = ee.Dictionary(points_label_list.zip(data_list_skew).flatten())
  return  ee.Feature(None, {'mean':prop_dict_mean, 'sdev':prop_dict_sdev, 'skew':prop_dict_skew})

out_fc = ee.FeatureCollection(means_fc_list.map(flatten_fn))

task = ee.batch.Export.table.toDrive(collection=out_fc,
                           description='NEX_Historical_Annual_Trends',
                           folder='GEE_Downloads')

task.start()




In [9]:
import statsmodels.api as sm
import numpy as np
import re

geeresFILE = '/content/drive/My Drive/GEE_Downloads/NEX_Historical_Annual_Trends_Agg.csv'
geedataFILE = '/content/drive/My Drive/GEE_Downloads/NEX_Historical_Annual_Trends.csv'
resFILE = '/content/drive/My Drive/Colab Notebooks/NEX_Historical_Annual_Trends_Results.csv'
dataFILE = '/content/drive/My Drive/Colab Notebooks/NEX_Historical_Annual_Trends_Data.csv'

"""For GHCN blocks"""
inFILE = '/content/drive/My Drive/Colab Notebooks/Script Input Files/GHCN_Historical_Annual_Results.csv'
"""For _CLINET blocks"""
#inFILE = '/content/drive/My Drive/Colab Notebooks/Script Input Files/US_CLIGEN_Coords.csv'

with open(inFILE) as f:
  lines = f.readlines()

stationIDjoin_Dict = {}
for line in lines[1:]:
  row = line.strip('\n').split(',')
  lon = row[1]
  lat = row[2]
  stationID = row[0]
  stationIDjoin_Dict[stationID] = [lon, lat]

with open(geeresFILE) as f:
  lines = f.readlines()

agg_data = {}
for line in lines[1:]:
  row = line.split(',')
  stationID = row[2]
  mean_agg = row[1]
  sdev_agg = row[3]
  skew_agg = row[4]
  agg_data[stationID] = [mean_agg, sdev_agg, skew_agg]

with open(geedataFILE) as f:
  lines = f.readlines()

data_dict = {'mean':{iD:[] for iD in stationIDjoin_Dict}, 'sdev':{iD:[] for iD in stationIDjoin_Dict}, 'skew':{iD:[] for iD in stationIDjoin_Dict}}
for line in lines[1:]:
  stat_dict_row = re.findall('{' + '(.+?)' + '}', line)
  for i, s in enumerate(['mean', 'sdev', 'skew']):
    row = stat_dict_row[i].strip('\n').split(',')
    for item in row:
      pair = item.strip().split('=')
      stationID = pair[0]
      number = pair[1]
      data_dict[s][stationID].append(number)

write_data = []

for iD in stationIDjoin_Dict:
  lon = stationIDjoin_Dict[iD][0]
  lat = stationIDjoin_Dict[iD][1]
  slopes = []
  for i, s in enumerate(['mean', 'sdev', 'skew']):
    y = [float(number) for number in data_dict[s][iD]]
    x = np.arange(0, 40)
    X = sm.add_constant(x)
    model = sm.OLS(y, X)
    results = model.fit()
    slope = results.params[1]
    slopes.append(slope)

  slope_means = str(slopes[0])
  slope_sdevs = str(slopes[1])
  slope_skews = str(slopes[2])
  mean = agg_data[iD][0]
  sdev = agg_data[iD][1]
  skew = agg_data[iD][2]

  write_data.append([iD, lon, lat, slope_means, mean, slope_sdevs, sdev, slope_skews, skew])


with open(resFILE, 'w') as fo:
  fo.write('stationID,x,y,slope_means,mean,slope_sdevs,sdev,slope_skews,skew\n')
  for row in write_data:
    fo.write(','.join(row) + '\n')

with open(dataFILE, 'w') as fo:
  hdr = ''
  for iD in stationIDjoin_Dict:
    hdr += ',{}_means,{}_sdevs,{}_skews'.format(iD, iD, iD)
  fo.write('year' + hdr + '\n')

  for i, yr in enumerate(list(range(1974, 2014))):
    data_str = str(yr)
    for iD in stationIDjoin_Dict:
      for s in ['mean', 'sdev', 'skew']:
        number_str = data_dict[s][iD][i]
        data_str = data_str + ',' + number_str
    data_str += '\n'
    fo.write(data_str)



